# Principle Component Analysis Using Python



[Course Module: Applied Data Analysis for Atmospheric Sciences Using Python]


## PCA with the MNIST dataset


----------------------------------

November 2018
Ting-Shuo Yo

# Outline

- MNIST 是一個手寫數字的圖像資料集，經常用於圖像辨識的基本範例。
- MNIST 資料集可至 [THE MNIST DATABASE](http://yann.lecun.com/exdb/mnist/)下載。
- 完整版的 MNIST 資料集包含 60000 個訓練圖片和 10000 個測試圖片，每個圖片大小是 28 * 28 像素。
- Python 的 scikit-learn 套件內建了簡易版的 [MNIST dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits)

|MNIST in sklearn|-|
|-----------|------------|
|Classes	|10|
|Samples per class	|~180|
|Samples total	|1797|
|Dimensionality	|64|
|Features	|integers 0-16|

首先，我們來讀取 scikit-learn 內建的 MNIST dataset，依據[使用手冊](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits)的說明，會提供一個 1797\*64 和一個 1797\*1 的 numpy.Array，前者是 1797 張 8\*8 的影像，而後者是每一張影像所寫的「數字」。

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import datasets

# The digits dataset
digits = datasets.load_digits()
print(digits.data.shape)
print(digits.target.shape)
print(pd.crosstab(digits.target,'count'))

接著，我們來看看資料集裡圖片的長相：

In [ ]:
# Create a 2*5 plots for each digit
fig, axes = plt.subplots(2, 5, sharex=True, sharey=True, squeeze=False)

# Loop through each class
for i in range(10):
    # Retrieve the first image of the given digit
    img = digits.data[(digits.target==i)][0]
    x = i // 5    # Put the digit to the right row/col
    y = i % 5
    axes[x,y].imshow(img.reshape(8,8), cmap='gray')
    axes[x,y].axis('off')
    axes[x,y].set_title(str(i))

所以，我們的 digits.data 資料集，是把每張 8\*8 像素的影像扁平化（[`numpy.flatten()`](https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.ndarray.flatten.html)），成為長度為 64 的向量，我們可以把每個像素想像成一個變項 $X_i$，如果我們對這個 1797\*64 的矩陣做 PCA，我們計算出的結果代表著什麼實際意義？

讓我們先看看各主成分的解釋變異量：

In [ ]:
pca_mnist = PCA(n_components=64)
pca_mnist.fit(digits.data)
# np.cumsum(X) will calculate the cumulative sum of X
plt.bar(range(64),pca_mnist.explained_variance_ratio_)
plt.plot(np.cumsum(pca_mnist.explained_variance_ratio_), 'r-')
plt.show()

我們可以很明顯的看到，與前面的幾個例子比較，手寫數字的影像資料，並不容易用少數的幾個主成分就解釋大部分的變異，至少要 13 個的主成分，才能解釋 80% 的變異量；要解釋90%的變異量則需要21個主成分。

In [ ]:
print(np.cumsum(pca_mnist.explained_variance_ratio_))

由於手寫數字的影像很容易透過視覺化來理解，恰好可以用來呈現「解釋 N% 的變異量」這件事情是什麼意思。在這裡我們需要用到一些數學，讓我們稍作解釋。

在 PCA 的介紹中提到，我們有一組觀測資料 $X_{M x N}$，包含了對 $N$ 個變數的 $M$ 次觀測，經過 PCA 之後我們找到一組 complete linear transform $W_{N x N}$，使得新座標軸 $Y = X \cdot W $，在 $Y$ 上資料的變異量由大至小排列。在 `sklearn.decomposition.PCA` 裡，$W$ 就是 `PCA.components_`，而原本資料在新座標軸的投影：

PCA.transform(X) = (X - PCA.mean_) $\cdot$ PCA.components_

然而，如果我們只保留前 $K$ 個主成分的資訊（$K < N$），那麼這個線性轉換就不是 complete transform，而重新將新座標軸的資料投影回原座標軸，就達到了「濾波」的效果。要用 scikit-learn 做到這件事情，基本上有兩種作法：

1. 在初始化 PCA 物件的時候，先指定 $K$ 或是「解釋變異量」：

`
pca = PCA(n_components=k)                        # or pca = PCA(0.90)
pca.fit(data)                                    # Calculate the transformation matrix
new_data = pca.transform(data)                   # Project the original data into the new coordinate
filtered_data = pca.inverse_transform(new_data)  # , and then project back
`

2. 計算完整的 PCA，取前 $K$ 個主成分，然後做矩陣運算：

`
pca = PCA()
pca.fit(data)                                                      # Calculate the transformation matrix
new_data = np.dot(data - pca.mean_, pca.components_[:k].T)         # Project the original data into the new coordinate
filtered_data = np.dot(new_data, pca.components_[:k]) + pca.mean_  # , and then project back
`


第一種方法比較簡單，但是當想要嘗試不同 $K$ 的效果時，每次更改 $K$ 就需要重新計算一次，在資料量大的時候效率會比較差。

接下來，我們拿數字 4 和 8 來做例子：

In [ ]:
# The original image of 4 and 8
img4 = digits.data[(digits.target==4)][0]
img8 = digits.data[(digits.target==8)][0]

# Fit the PCA
pca = PCA()
pca.fit(digits.data)

# Define a function of filter for repeated use
def filter_pca(x, pca, k):
    '''
    x  : numpy array, the original data with shape(N,1).
    pca: object, the fitted PCA with n_components = N.
    k  : integer, the number of PCs to use.
    '''
    x_new = pca.transform(x)
    x_filtered = np.dot(x_new[:,:k], pca.components_[:k]) + pca.mean_
    return(x_filtered)


In [ ]:
# As shown in above, k required to explain 80%, 90%, 95%, and 99% variance are:
ks = [13, 21, 29, 41, 64]
ps = ['80%', '90%', '95%', '99%', '100%']

# Create a 2*5 plots for each digit
fig, axes = plt.subplots(2, 5, squeeze=False)

# Loop through each class
for i in range(len(ks)):
    # Calculate filtered images
    img4f = filter_pca(img4.reshape(1,64), pca, ks[i])
    img8f = filter_pca(img8.reshape(1,64), pca, ks[i])
    # Plot
    axes[0,i].imshow(img4f.reshape(8,8), cmap='gray')
    axes[0,i].axis('off')
    axes[0,i].set_title(ps[i])
    axes[1,i].imshow(img8f.reshape(8,8), cmap='gray')
    axes[1,i].axis('off')
    
plt.tight_layout()
plt.show()

## PCs of Hand-Written Digits

從上面的例子，我們看到透過使用不同數目的主成分可以達到 filter 的效果，那麼這些主成分又是些什麼？

In [ ]:
# Create a 2*5 plots for each digit
fig, axes = plt.subplots(2, 6, sharex=True, sharey=True, squeeze=False)

# Loop through each class
for i in range(12):
    # Retrieve the first image of the given digit
    pc = pca.components_[i,:]
    x = i // 6    # Put the digit to the right row/col
    y = i % 6
    axes[x,y].imshow(pc.reshape(8,8), cmap='gray')
    axes[x,y].axis('off')
    axes[x,y].set_title('PC'+str(i+1))
plt.tight_layout()
plt.show()

很顯然，手寫數字影像的主成分，看起來不像是任何單一的數字，但是我們可以理解成 0～9 的數字在 8\*8 像素平面上的「特徵」（feature），而前13個特徵幾乎就可以組合出可以便是的數字了。

如果我們把「手寫數字的影像」換成再分析資料的某個特性層，例如 850hPa 的溫度場，那麼 PCA 結果的詮釋方法，就與這個例子非常類似。事實上，這個方法在地球物理領域叫做 [Emperical Orthogonal Functions (EOF) Analysis](https://en.wikipedia.org/wiki/Empirical_orthogonal_functions) ，早期很多大氣的低頻震盪都是透過這個工具找到的。

我們接下來會用赤道太平洋的海溫資料做例子，讓各位可以重新發現一次**聖嬰現象**，不過在進行這個分析工作之前，我們需要先介紹另一個主題：把資料畫在地圖上。